#### TELECOM CHURN MANAGEMENT - SEGMENTATION - CLASIFICATION ( DATA CLEANING - CHURNED CUSTOMERS )

    #1. Understand the data and perform the data analysis on churned customers' data

    A. Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm
import scipy.stats as stats
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor
from patsy import dmatrices
import pandas_profiling
%matplotlib inline

C:\Users\lenovo\AppData\Local\Continuum\anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\lenovo\AppData\Local\Continuum\anaconda2\lib\site-packages\pandas_profiling\base.py:20: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "C:\Users\lenovo\AppData\Local\Continuum\anaconda2\lib\runpy.py", line 174, in _run_module_as_main
    "__main__

    B. Importing data 

In [2]:
telecom_churn = pd.read_csv("telecom_churn.csv")

    C. PURIFYING DATA FOR FURTHER ANALYSIS

    ************************************** CHURN CUSTOMERS ****************************************************

In [3]:
telecom_churn.head()

,SITE_ACCOUNT_NUMBER,ACCOUNT_STATUS,FIRST_COMMUNICATION_DATE,CHURN_DATE,UPGRADE_DATE,SERVICE_PLAN,MODEL,SALES_CHANNEL,DIRECT_INDIRECT_CHANNEL,SALES_SOURCE,...,SITE_TYPE_DESC,WARRANTY_NAME,MOST_RECENT_SALES_CHANNEL,GENDER,DATE_OF_BIRTH,AGE,INCOME,MARITAL_STATUS,PRESENCE_OF_CHILDREN,COMPUTER_OWNER
0,AMR-53205840,Closed,18-01-2002,02-04-2006,03-01-2004,Professional,DW6000,NaN,Indirect,Perfect 10,...,Broadband Remote,NaN,Dealer-Generated Sale,2.0,27-10-1964,48,0,NaN,NaN,N
1,AMR-53383136,Closed,02-01-2002,09-07-2006,28-07-2004,SO,DW6000,Retail/Others,Indirect,ValueElectronics,...,Broadband Remote,NaN,National Sales,NaN,22-10-1965,47,5,0,0.0,Y
2,AMR-53608585,Closed,06-01-2002,30-12-2008,27-04-2005,NaN,DW6000,Retail/Others,Indirect,TAG,...,Broadband Remote,NaN,Direct,2.0,20-03-1951,61,8,1,0.0,Y
3,AMR-53610590,Closed,05-01-2002,19-08-2004,NaN,SO,DW4000,NaN,Indirect,TAG,...,Broadband Remote,NaN,Direct,1.0,27-03-1948,64,0,NaN,NaN,Y
4,AMR-53612154,Closed,05-01-2002,17-08-2006,12-11-2005,Professional,DW7000,Retail/Others,Indirect,Perfect 10,...,Broadband Remote,NaN,Dealer-Generated Sale,1.0,25-08-1956,56,6,0,0.0,N


In [4]:
pandas_profiling.ProfileReport(telecom_churn)

Number of variables,30
Number of observations,10000
Total Missing (%),6.7%
Total size in memory,2.3 MiB
Average record size in memory,240.0 B
Numeric,2
Categorical,27
Date,0
Text (Unique),1
Rejected,0
Distinct count,2


    Dropping variables with high cardinality and non retrospective - 

In [5]:
telecom_churn.drop(['ACCOUNT_STATUS' , 'ADDRESS', 'CITY', 'COMPANY_SOURCE_NAME', 'DATE_OF_BIRTH', 'EMAIL_ADDRESS', 'FIRST_COMMUNICATION_DATE', 'FIRST_NAME', 'LAST_NAME', 'ZIPCODE','CHURN_DATE','STATE', 'SALES_SOURCE'], axis=1, inplace = True)

In [6]:
telecom_churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
SITE_ACCOUNT_NUMBER          10000 non-null object
UPGRADE_DATE                 9121 non-null object
SERVICE_PLAN                 8518 non-null object
MODEL                        9290 non-null object
SALES_CHANNEL                7398 non-null object
DIRECT_INDIRECT_CHANNEL      9216 non-null object
FEEDBACK                     7972 non-null object
SATELLITE                    8987 non-null object
SITE_TYPE_DESC               9937 non-null object
WARRANTY_NAME                9328 non-null object
MOST_RECENT_SALES_CHANNEL    9995 non-null object
GENDER                       8762 non-null float64
AGE                          10000 non-null object
INCOME                       10000 non-null object
MARITAL_STATUS               6388 non-null object
PRESENCE_OF_CHILDREN         8748 non-null float64
COMPUTER_OWNER               6757 non-null object
dtypes: float64(2), object(15)
memory

In [7]:
telecom_churn.isnull().any()

SITE_ACCOUNT_NUMBER          False
UPGRADE_DATE                  True
SERVICE_PLAN                  True
MODEL                         True
SALES_CHANNEL                 True
DIRECT_INDIRECT_CHANNEL       True
FEEDBACK                      True
SATELLITE                     True
SITE_TYPE_DESC                True
WARRANTY_NAME                 True
MOST_RECENT_SALES_CHANNEL     True
GENDER                        True
AGE                          False
INCOME                       False
MARITAL_STATUS                True
PRESENCE_OF_CHILDREN          True
COMPUTER_OWNER                True
dtype: bool

    Replacing #REF and NULL of Categorical variables as 'NA'

In [8]:
telecom_churn[['COMPUTER_OWNER', 'DIRECT_INDIRECT_CHANNEL', 'FEEDBACK', 'SALES_CHANNEL','SITE_TYPE_DESC' ]] = telecom_churn[['COMPUTER_OWNER', 'DIRECT_INDIRECT_CHANNEL', 'FEEDBACK', 'SALES_CHANNEL', 'SITE_TYPE_DESC']].replace(np.nan, 'NA', regex=True)

In [9]:
telecom_churn[['COMPUTER_OWNER', 'DIRECT_INDIRECT_CHANNEL', 'FEEDBACK', 'SALES_CHANNEL','SITE_TYPE_DESC' ]] = telecom_churn[['COMPUTER_OWNER', 'DIRECT_INDIRECT_CHANNEL', 'FEEDBACK', 'SALES_CHANNEL','SITE_TYPE_DESC' ]].replace('REF!', 'NA', regex=True)

    Replacing #REF and NULL of Numeric Varaibles as '9' and converting the variable type to int64
    9 - implies UNKNOWN

In [10]:
telecom_churn[['GENDER', 'MARITAL_STATUS', 'PRESENCE_OF_CHILDREN' ]] = telecom_churn[['GENDER', 'MARITAL_STATUS', 'PRESENCE_OF_CHILDREN']].replace('#REF!', '9', regex=True)

In [11]:
telecom_churn[['GENDER', 'MARITAL_STATUS', 'PRESENCE_OF_CHILDREN' ]] = telecom_churn[['GENDER', 'MARITAL_STATUS', 'PRESENCE_OF_CHILDREN' ]].replace(np.nan, '9', regex=True)

In [12]:
telecom_churn[['AGE' ]] = telecom_churn[['AGE']].replace('#REF!', '0', regex=True)

In [13]:
MEAN_AGE_CHURN = telecom_churn['AGE'].mean()
telecom_churn['AGE']=telecom_churn.AGE.mask(telecom_churn.AGE == 0,MEAN_AGE_CHURN)

In [14]:
telecom_churn[['GENDER', 'MARITAL_STATUS', 'PRESENCE_OF_CHILDREN','AGE' ]] = telecom_churn[['GENDER', 'MARITAL_STATUS', 'PRESENCE_OF_CHILDREN','AGE' ]].astype('int64')

In [15]:
telecom_churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
SITE_ACCOUNT_NUMBER          10000 non-null object
UPGRADE_DATE                 9121 non-null object
SERVICE_PLAN                 8518 non-null object
MODEL                        9290 non-null object
SALES_CHANNEL                10000 non-null object
DIRECT_INDIRECT_CHANNEL      10000 non-null object
FEEDBACK                     10000 non-null object
SATELLITE                    8987 non-null object
SITE_TYPE_DESC               10000 non-null object
WARRANTY_NAME                9328 non-null object
MOST_RECENT_SALES_CHANNEL    9995 non-null object
GENDER                       10000 non-null int64
AGE                          10000 non-null int64
INCOME                       10000 non-null object
MARITAL_STATUS               10000 non-null int64
PRESENCE_OF_CHILDREN         10000 non-null int64
COMPUTER_OWNER               10000 non-null object
dtypes: int64(4), object(13)
memory

In [16]:
telecom_churn.isnull().any()

SITE_ACCOUNT_NUMBER          False
UPGRADE_DATE                  True
SERVICE_PLAN                  True
MODEL                         True
SALES_CHANNEL                False
DIRECT_INDIRECT_CHANNEL      False
FEEDBACK                     False
SATELLITE                     True
SITE_TYPE_DESC               False
WARRANTY_NAME                 True
MOST_RECENT_SALES_CHANNEL     True
GENDER                       False
AGE                          False
INCOME                       False
MARITAL_STATUS               False
PRESENCE_OF_CHILDREN         False
COMPUTER_OWNER               False
dtype: bool

    Treating Catergorical variables with large sets of unique values

    STEP 01 - Missing value treatment

In [17]:
telecom_churn[['SERVICE_PLAN', 'MODEL', 'SATELLITE', 'WARRANTY_NAME', 'MOST_RECENT_SALES_CHANNEL' ]] = telecom_churn[['SERVICE_PLAN', 'MODEL',  'SATELLITE', 'WARRANTY_NAME', 'MOST_RECENT_SALES_CHANNEL']].replace('#REF!', 'NA', regex=True)

In [18]:
telecom_churn[['SERVICE_PLAN', 'MODEL',  'SATELLITE', 'WARRANTY_NAME', 'MOST_RECENT_SALES_CHANNEL' ]] = telecom_churn[['SERVICE_PLAN', 'MODEL',  'SATELLITE', 'WARRANTY_NAME', 'MOST_RECENT_SALES_CHANNEL']].replace(np.nan, 'NA', regex=True)

    STEP 02 - Working with column names and column values
              Special consideration on unwanted spaces and squeezing vvariables wherever required

###### COLUMN NAME - SERVICE_PLAN

In [19]:
telecom_churn.groupby('SERVICE_PLAN')['SITE_ACCOUNT_NUMBER'].nunique()

SERVICE_PLAN
Basic                 623
Basic_Ltd             666
Elite                1795
Home                 1122
NA                   1482
Power15               624
Pro                   555
Professional          780
Professional Plus     505
Proplus               664
SO                   1184
Name: SITE_ACCOUNT_NUMBER, dtype: int64

In [20]:
telecom_churn['SERVICE_PLAN_CAT'] = telecom_churn['SERVICE_PLAN' ].str.strip()

In [21]:
telecom_churn.groupby('SERVICE_PLAN_CAT')['SITE_ACCOUNT_NUMBER'].nunique()

SERVICE_PLAN_CAT
Basic                 623
Basic_Ltd             666
Elite                1795
Home                 1122
NA                   1482
Power15               624
Pro                   555
Professional          780
Professional Plus     505
Proplus               664
SO                   1184
Name: SITE_ACCOUNT_NUMBER, dtype: int64

In [22]:
telecom_churn.drop('SERVICE_PLAN', axis =1 , inplace = True)

##### COLUMN NAME - MODEL

In [23]:
telecom_churn.groupby('MODEL')['SITE_ACCOUNT_NUMBER'].nunique()

MODEL
DW4000     1618
DW6000     1380
DW7000     1500
HN7000S    1899
HN9000     2850
NA          710
UNK          43
Name: SITE_ACCOUNT_NUMBER, dtype: int64

In [24]:
telecom_churn['MODEL_CAT'] = telecom_churn['MODEL' ].str.strip()

In [25]:
telecom_churn.drop('MODEL', axis = 1, inplace = True)

##### COLUMN NAME - MOST_RECENT_SALES_CHANNEL
    - Creating new categorical variable MOST_RECENT_SALES_CHANNEL_CAT
    - Replacing "DW6000 Upgrade", "DW7000 Upgrade" with "DW6K Upgrade", "DW7K Upgrade"
    - Replacing "HN7000 Upgrade", "HN9000 Upgrade" with "HN9K Upgrade"

In [26]:
telecom_churn.groupby('MOST_RECENT_SALES_CHANNEL')['SITE_ACCOUNT_NUMBER'].nunique()

MOST_RECENT_SALES_CHANNEL
0                          46
Business Edition           20
DPC Classic upgrade        74
DW6000 Upgrade            137
DW7000 Upgrade             55
Dealer - Direct Order     623
Dealer-Generated Sale     205
Direct                   4175
Distributor Sales Age      77
EXPRESS REPAIR BASIC       11
HN7000 Upgrade            166
Hughes Sales Agents      3825
NA                         21
National Sales            112
Retail                     97
Retail - Direct Order      31
UNKNOWN                    34
Used Equipment            291
Name: SITE_ACCOUNT_NUMBER, dtype: int64

In [27]:
telecom_churn['MOST_RECENT_SALES_CHANNEL_CAT'] = telecom_churn['MOST_RECENT_SALES_CHANNEL'].str.strip()

In [28]:
telecom_churn[['MOST_RECENT_SALES_CHANNEL_CAT']] = telecom_churn[['MOST_RECENT_SALES_CHANNEL_CAT']].replace(["DW6000 Upgrade"], "DW6K Upgrade", regex=True)

In [29]:
telecom_churn[['MOST_RECENT_SALES_CHANNEL_CAT']] = telecom_churn[['MOST_RECENT_SALES_CHANNEL_CAT']].replace(['DW7000 Upgrade'], 'DW7K Upgrade', regex=True)

In [30]:
telecom_churn[['MOST_RECENT_SALES_CHANNEL_CAT']] = telecom_churn[['MOST_RECENT_SALES_CHANNEL_CAT']].replace(['HN7000 Upgrade'], 'HN7K Upgrade', regex=True)

In [31]:
telecom_churn[['MOST_RECENT_SALES_CHANNEL_CAT']] = telecom_churn[['MOST_RECENT_SALES_CHANNEL_CAT']].replace(['0','UNKNOWN'], 'NA', regex=True)

In [32]:
telecom_churn.groupby('MOST_RECENT_SALES_CHANNEL_CAT')['SITE_ACCOUNT_NUMBER'].nunique()

MOST_RECENT_SALES_CHANNEL_CAT
Business Edition           20
DPC Classic upgrade        74
DW6K Upgrade              137
DW7K Upgrade               55
Dealer - Direct Order     623
Dealer-Generated Sale     205
Direct                   4175
Distributor Sales Age      77
EXPRESS REPAIR BASIC       11
HN7K Upgrade              166
Hughes Sales Agents      3825
NA                        101
National Sales            112
Retail                     97
Retail - Direct Order      31
Used Equipment            291
Name: SITE_ACCOUNT_NUMBER, dtype: int64

In [33]:
telecom_churn.drop('MOST_RECENT_SALES_CHANNEL', axis =1, inplace = True)

##### COLUMN NAME - SATELLITE
    Squeezing the varaibles as VER and HOR type


In [34]:
telecom_churn.groupby('SATELLITE')['SITE_ACCOUNT_NUMBER'].nunique()

SATELLITE
AM6-HOR-21K     504
AM9-HOR-17K     336
AMC-VER-5K      418
G16-VER-15K     521
G26-HOR-14K     475
G28-HOR-10K     428
G28-HOR-16K     517
G3C-VER-24K     977
HZN-HOR-8K      601
HZN-VER-23K     427
NA             1013
SM5-VER-7K      518
SM6-HOR-10K     807
SPACEWAY3      2458
Name: SITE_ACCOUNT_NUMBER, dtype: int64

In [35]:
telecom_churn['SATELLITE_CAT'] = telecom_churn['SATELLITE'].str.strip()

In [36]:
telecom_churn[['SATELLITE_CAT']] = telecom_churn[['SATELLITE_CAT']].replace(["AM6-HOR-21K","AM32-HOR-7K","AM9-HOR-17K"], "AM-HOR-SERIES", regex=True)

In [37]:
telecom_churn[['SATELLITE_CAT']] = telecom_churn[['SATELLITE_CAT']].replace(["AMC-VER-5K"], "AM-VER-SERIES", regex=True)

In [38]:
telecom_churn[['SATELLITE_CAT']] = telecom_churn[['SATELLITE_CAT']].replace(["G26-HOR-14K","G28-HOR-10K","G28-HOR-16K"], "G-HOR-SERIES", regex=True)

In [39]:
telecom_churn[['SATELLITE_CAT']] = telecom_churn[['SATELLITE_CAT']].replace(["G16-VER-15K","G3C-VER-24K"], "G-VER-SERIES", regex=True)

In [40]:
telecom_churn[['SATELLITE_CAT']] = telecom_churn[['SATELLITE_CAT']].replace(["HZN-HOR-8K"], "HZN-HOR-SERIES", regex=True)

In [41]:
telecom_churn[['SATELLITE_CAT']] = telecom_churn[['SATELLITE_CAT']].replace(["HZN-VER-23K"], "HZN-VER-SERIES", regex=True)

In [42]:
telecom_churn[['SATELLITE_CAT']] = telecom_churn[['SATELLITE_CAT']].replace(["SM5-VER-7K"], "SM5-VER-SERIES", regex=True)

In [43]:
telecom_churn[['SATELLITE_CAT']] = telecom_churn[['SATELLITE_CAT']].replace(["SM6-HOR-10K"], "SM6-HOR-SERIES", regex=True)

In [44]:
telecom_churn.groupby('SATELLITE_CAT')['SITE_ACCOUNT_NUMBER'].nunique()

SATELLITE_CAT
AM-HOR-SERIES      840
AM-VER-SERIES      418
G-HOR-SERIES      1420
G-VER-SERIES      1498
HZN-HOR-SERIES     601
HZN-VER-SERIES     427
NA                1013
SM5-VER-SERIES     518
SM6-HOR-SERIES     807
SPACEWAY3         2458
Name: SITE_ACCOUNT_NUMBER, dtype: int64

In [45]:
telecom_churn.drop('SATELLITE', axis =1 , inplace = True)

##### COLUMN NAME - WARRANTY_NAME

In [46]:
telecom_churn.groupby('WARRANTY_NAME')['SITE_ACCOUNT_NUMBER'].nunique()

WARRANTY_NAME
0                                               6116
Broadband Remote                                  39
EXPRESS REPAIR BASIC                            2472
EXPRESS REPAIR BUSINESS WITH TERM                 23
EXPRESS REPAIR HOME                               14
EXPRESS REPAIR HOME W/O COMMITMENT WITH TERM      10
EXPRESS REPAIR HOME WITH TERM                    303
NA                                               681
SRVC ASSUR PLUS NEW CUST                          28
SRVC ASSUR STD NEW CUST                          298
Spaceway                                          16
Name: SITE_ACCOUNT_NUMBER, dtype: int64

In [47]:
telecom_churn['WARRANTY_NAME_CAT'] = telecom_churn['WARRANTY_NAME'].str.strip()

In [48]:
telecom_churn[['WARRANTY_NAME_CAT']] = telecom_churn[['WARRANTY_NAME_CAT']].replace(["0"], "NA", regex=True)

In [49]:
telecom_churn.groupby('WARRANTY_NAME_CAT')['SITE_ACCOUNT_NUMBER'].nunique()

WARRANTY_NAME_CAT
Broadband Remote                                  39
EXPRESS REPAIR BASIC                            2472
EXPRESS REPAIR BUSINESS WITH TERM                 23
EXPRESS REPAIR HOME                               14
EXPRESS REPAIR HOME W/O COMMITMENT WITH TERM      10
EXPRESS REPAIR HOME WITH TERM                    303
NA                                              6797
SRVC ASSUR PLUS NEW CUST                          28
SRVC ASSUR STD NEW CUST                          298
Spaceway                                          16
Name: SITE_ACCOUNT_NUMBER, dtype: int64

In [50]:
telecom_churn.drop('WARRANTY_NAME', axis =1, inplace = True)

    Treating NULL value of INCOME with median (max repeatitive value) for active customers

In [51]:
# 0 - is the max repleatative value for active customers
telecom_churn.groupby('INCOME')['SITE_ACCOUNT_NUMBER'].nunique()

INCOME
0    2326
1     228
2     383
3    1440
4    1171
5    1175
6    1103
7     421
8     379
9     234
A     551
B     138
C     179
D     272
Name: SITE_ACCOUNT_NUMBER, dtype: int64

In [52]:
telecom_churn[['INCOME' ]] = telecom_churn[['INCOME']].replace('#REF!', '0', regex=True)

In [53]:
telecom_churn[['INCOME' ]] = telecom_churn[['INCOME']].replace(np.nan, '0', regex=True)

    D. Creating new KPIs for active customers' dataset
    - CHURN_IND 0 implying NOT churned, 1 implying churned
    - UPGRADE_IND 0 implying NOT upgraded, 1 implying upgraded
      (IF UPGRADE_DATE is availible - it implies upgraded)

In [54]:
telecom_churn['CHURN_IND'] = 1

In [55]:
telecom_churn['UPGRADE_IND'] = telecom_churn['UPGRADE_DATE'].str.strip()

In [56]:
telecom_churn[['UPGRADE_IND' ]] = telecom_churn[['UPGRADE_IND']].replace(np.nan, '0', regex=True)

In [57]:
telecom_churn.loc[telecom_churn['UPGRADE_IND'] <> '0', 'UPGRADE_IND'] = '1'

In [58]:
telecom_churn.groupby('UPGRADE_IND')['SITE_ACCOUNT_NUMBER'].nunique()

UPGRADE_IND
0     879
1    9121
Name: SITE_ACCOUNT_NUMBER, dtype: int64

In [59]:
telecom_churn.drop('UPGRADE_DATE', axis =1, inplace = True)

In [60]:
telecom_churn['UPGRADE_IND'] = telecom_churn['UPGRADE_IND'].astype('int64')

In [61]:
telecom_churn.isnull().any()

SITE_ACCOUNT_NUMBER              False
SALES_CHANNEL                    False
DIRECT_INDIRECT_CHANNEL          False
FEEDBACK                         False
SITE_TYPE_DESC                   False
GENDER                           False
AGE                              False
INCOME                           False
MARITAL_STATUS                   False
PRESENCE_OF_CHILDREN             False
COMPUTER_OWNER                   False
SERVICE_PLAN_CAT                 False
MODEL_CAT                        False
MOST_RECENT_SALES_CHANNEL_CAT    False
SATELLITE_CAT                    False
WARRANTY_NAME_CAT                False
CHURN_IND                        False
UPGRADE_IND                      False
dtype: bool

In [62]:
telecom_churn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
SITE_ACCOUNT_NUMBER              10000 non-null object
SALES_CHANNEL                    10000 non-null object
DIRECT_INDIRECT_CHANNEL          10000 non-null object
FEEDBACK                         10000 non-null object
SITE_TYPE_DESC                   10000 non-null object
GENDER                           10000 non-null int64
AGE                              10000 non-null int64
INCOME                           10000 non-null object
MARITAL_STATUS                   10000 non-null int64
PRESENCE_OF_CHILDREN             10000 non-null int64
COMPUTER_OWNER                   10000 non-null object
SERVICE_PLAN_CAT                 10000 non-null object
MODEL_CAT                        10000 non-null object
MOST_RECENT_SALES_CHANNEL_CAT    10000 non-null object
SATELLITE_CAT                    10000 non-null object
WARRANTY_NAME_CAT                10000 non-null object
CHURN_IND       

In [63]:
telecom_churn.to_csv("telecom_churn_cleaned.csv", index= False)

In [64]:
pandas_profiling.ProfileReport(telecom_churn)

Number of variables,18
Number of observations,10000
Total Missing (%),0.0%
Total size in memory,1.4 MiB
Average record size in memory,144.0 B
Numeric,5
Categorical,11
Date,0
Text (Unique),1
Rejected,1
Distinct count,66


    ************************************ END OF PART 2 ****************************************************